# Procesamiento de lenguaje - Texto descriptivo contratos

In [15]:
# IMPORTS
import pandas as pd
import os

In [16]:
# RUTAS 
BASE_PATH = r"C:\Users\POTENCIA\OneDrive - POTENCIA\Documents\TAREA_ENTIDADES"
DATAPATH = os.path.join(BASE_PATH, "data", "02_text")
FILTERPATH = os.path.join(BASE_PATH, "data", "01_filter")
os.makedirs(DATAPATH, exist_ok=True)
os.makedirs(FILTERPATH, exist_ok=True)

# Data
df = pd.read_excel("../data/01_filter/SECOP_filtro_contrato.xlsx")

In [17]:
# Distinción por familias de categorias

mantenimiento = df[df["codigo_familia_UNSPSC"] == 7210]
print(f"Mantenimiento y reparaciones {mantenimiento.shape}")

pesada = df[df["codigo_familia_UNSPSC"] == 7214]
print(f"Infraestructura pesada {pesada.shape}")

especial = df[df["codigo_familia_UNSPSC"] == 7215]
print(f"Infraestructura especializada {especial.shape}")

Mantenimiento y reparaciones (3641, 29)
Infraestructura pesada (2384, 29)
Infraestructura especializada (1015, 29)


## Estandarización de texto

In [18]:
# Estado 1 - Texto crudo
texto = df['descripcion_std']
conteo = texto.value_counts()
print(f"Coincidencias {conteo.shape}")
out = os.path.join(DATAPATH, 'TEXTO_1.xlsx')
conteo.to_excel(out) 


Coincidencias (6722,)


In [19]:
# Estado 2 - Determinar información clave
texto_1 = texto.copy()

# Diccionario de palabras a omitir
init_words = [
            'el', 'la', 'las', 'los', 'a', 'de', 'del', 'para', 

            'bs', 'c', 'e', 'es', 'sg', 'srt', 'mr', 'd', 'dt', 'nsa',
            's', 'sa', 'se', 
            'lote', 'rmtc', 'rstc', 'rtvc', 'rvlc', 'no',
            'spa', 'oap', 'sol', 'nr', 'ce', 'srn', 'srnc',

            'ranc', 'ratc', 'rcnc', 'fun', 'fortis', 
            'amf', 'amfis', 'sm', 'dtnsa', 

            'realizar', 'rrealizar', 'realizacion', 'ealizar',
            'actividades', 'grupo', 'servicio', 'servicios',
            'ejecucion', 'ejecutar', 'esfuerzos', 
            'prestar', 'global', 'segunda', 'fase',

            'obra', 'obras', 'publica', 'civil', 'civiles', 'complementarias',

            'llevar a cabo', 'mano', 'necesarias', 'segunda',

            'mediante', 'por', 'sistema', 'sin', 'formula', 
            'ajuste', 'reajuste', 'todo', 'costo',

            'aunar', 'anuar', 'unar', 'esfuerzo', 'y' , 'apoyo mutuo', 
            'precios', 'precio', 'unitario', 'unitarios', 'fijo', 'fijos',

            'contratar', 'bajo', 
              ]

def limpiar_inicio(sentence, palabras_ini):
    if not isinstance(sentence, str):
        return sentence
    
    sentence = sentence.strip()
    # eliminar todas las palabras iniciales consecutivas que estén en la lista
    while True:
        partes = sentence.split(maxsplit=1)
        if partes and partes[0].lower() in palabras_ini:
            sentence = partes[1] if len(partes) > 1 else ""
        else:
            break
    return sentence

texto_1 = texto_1.apply(lambda x: limpiar_inicio(x, init_words))
conteo_1 = texto_1.value_counts()
print(f"Coincidencias {conteo_1.shape}")
out = os.path.join(DATAPATH, 'TEXTO_2.xlsx')
conteo_1.to_excel(out) 

Coincidencias (6692,)


In [20]:
# LIMPIEZA - Aplicar estructutración de texto
df['descripcion_std'] = df['descripcion_std'].apply(lambda x: limpiar_inicio(x, init_words))

## Categorización por acción realizada

In [21]:
# FUNCION - Categorias de accion

categoria = {
    'Adecuacion': {'adecuacion', 'adecuar', 'adecuaciones', 'acondicionamiento', 'habilitar'},
    'Construccion': {'construccion', 'construir', 'construcciones', 'reconstruir', 'reconstruccion',
                     'demoler', 'demolicion', 'desmontaje', 'desmonte', 'desmontar', 'instalacion'},
    'Mantenimiento': {'mantenimiento', 'mantener'},
    'Reparacion': {'reparacion', 'reparaciones', 'reparar', 'rehabilitar', 'recuperacion', 'rehabilitacion', 'restauracion'},
    'Atencion': {'atencion', 'atender'},
    'Mejoramiento': {'mejoramiento', 'mejorar', 'remodelar', 'remodelacion', 'ampliar', 'ampliacion', 'modernizacion'},
}

def clasificar(texto):
    if not isinstance(texto, str):
        return {"todas": "Otros", "principal": "Otros"}
    
    texto = texto.lower()
    encontradas = []
    
    for cat, palabras in categoria.items():
        for palabra in palabras:
            if palabra in texto:
                encontradas.append(cat)
                break  # evita duplicar la misma categoría
    
    if not encontradas:
        return {"todas": "Otros", "principal": "Otros"}
    
    return {
        "todas": "; ".join(encontradas),   # todas las categorías encontradas
        "principal": encontradas[0]        # primera coincidencia
    }

# Aplicar a una columna del DataFrame
df[["acciones", "tipo_accion"]] = df["descripcion_std"].apply(
    lambda x: pd.Series(clasificar(x))
)

cate = df[['descripcion_std', 'acciones', 'tipo_accion']]
out = os.path.join(DATAPATH, 'Clasificacion_1.xlsx')
cate.to_excel(out, index=False) 

out = os.path.join(FILTERPATH, 'SECOP_cat_accion.xlsx')
df.to_excel(out, index=False) 

In [23]:
tabla = (df["tipo_accion"].value_counts(dropna=False).rename("conteo").reset_index())
tabla["porcentaje"] = (tabla["conteo"] / tabla["conteo"].sum() * 100).round(2)
tabla.columns = ["tipo_accion", "valor_neto", "porcentaje"]
display(tabla)

,tipo_accion,valor_neto,porcentaje
0,Mantenimiento,3271,42.15
1,Adecuacion,1982,25.54
2,Construccion,1499,19.31
3,Mejoramiento,404,5.21
4,Otros,398,5.13
5,Reparacion,136,1.75
6,Atencion,71,0.91


## Categorización por rubro

In [28]:
# FUNCIÓN - Categorización por rubro

category = cate.copy()

categorias = {
    "Ambiental": ["ambiente","ambiental","ecologico","ecologica","sostenible","reforestacion","biodiversidad","agua","aguas"],
    "Servicios publicos": ["servicio publico","servicios publicos","acueducto","alcantarillado","gas","luz","energia","agua potable","residuos solidos","aseo"],
    "Transporte": ["transporte","movilidad","terminal","bus","buses","metro","bicicleta","bicicletas"],
    "Agro": ["agro","agricola","agricolas","ganaderia","rural","campo","siembra","riego","agropecuario","agropecuaria"],
    "Aeropuerto": ["aeropuerto","aeropuertos","aviacion","aereo","aerea"],
    "Deporte": ["deporte","deportes","polideportivo","polideportivos","cancha","canchas","coliseo","recreacion","gimnasio","gimnasios"],
    "Educacion": ["educacion","colegio","colegios","escuela","escuelas","universidad","universidades","aula","aulas","institucion educativa","instituciones educativas"],
    "Energia": ["energia","electrica","electrico","solar","panel","paneles","luminaria","luminarias","electricidad"],
    "Infraestructura urbana": ["infraestructura urbana","anden","andenes","urbanismo","espacio publico","espacios publicos","barrio","barrios","centro urbano","centros urbanos"],
    "Vias": ["via","vias","carretera","carreteras","camino","caminos","pavimentacion","vial","viales","puente","puentes","interseccion","intersecciones","rotonda","rotondas"],
    "Parque": ["parque","parques","zona verde","zonas verdes","recreativo","recreativos","jardin","jardines","plazoleta","plazoletas"],
    "Playa": ["playa","playas","costero","costera","maritimo","maritima"],
    "Puerto": ["puerto","puertos","muelle","muelles","embarcadero","embarcaderos","fluvial","fluviales"],
    "Rio": ["rio","rios","cuenca","cuencas","canal","canales","afluente","afluentes"],
    "Tren": ["tren","trenes","ferrocarril","ferrocarriles","ferroviario","ferroviaria"],
    "Turismo": ["turismo","turistico","turistica","atractivo","atractivos","visitante","visitantes","cultural","culturales"],
    "Vivienda": ["vivienda","viviendas","habitacional","habitacionales","residencia","residencias","urbanizacion","urbanizaciones","mejoramiento de vivienda","mejoramientos de vivienda"]
}

def clasificar_tema(texto):
    if not isinstance(texto, str):
        return {"categorias": "Otros", "tipo_categoria": "Otros"}
    
    texto = texto.lower()
    encontradas = []
    posiciones = {}
    
    for cat, palabras in categorias.items():
        for palabra in palabras:
            pos = texto.find(palabra)
            if pos != -1:
                encontradas.append(cat)
                # guardamos la posición de la primera aparición
                if cat not in posiciones or pos < posiciones[cat]:
                    posiciones[cat] = pos
                break  # evita duplicar la misma categoría
    
    if not encontradas:
        return {"categorias": "Otros", "tipo_categoria": "Otros"}
    
    # todas las categorias encontradas
    todas = "; ".join(encontradas)
    # categoría principal = la que aparece primero en el texto
    principal = min(posiciones, key=posiciones.get)
    
    return {"categorias": todas, "tipo_categoria": principal}

# Aplicar a una columna del DataFrame
df[["acciones", "tipo_accion"]] = df["descripcion_std"].apply(
    lambda x: pd.Series(clasificar(x))
)

df[["categorias", "tipo_categoria"]] = df["descripcion_std"].apply(
    lambda x: pd.Series(clasificar_tema(x))
)
cat = df[['descripcion_std', 'acciones', 'tipo_accion', 'categorias', 'tipo_categoria']]
out = os.path.join(DATAPATH, 'Categoria.xlsx')
cat.to_excel(out, index=False) 

out = os.path.join(FILTERPATH, 'SECOP_CATEGORIAS.xlsx')
df.to_excel(out, index=False) 

In [33]:
tabla = (df["tipo_categoria"].value_counts(dropna=False).rename("conteo").reset_index())
tabla["porcentaje"] = (tabla["conteo"] / tabla["conteo"].sum() * 100).round(2)
tabla.columns = ["tipo_categoria", "conteo", "porcentaje"]
display(tabla)

,tipo_categoria,conteo,porcentaje
0,Otros,1832,23.61
1,Rio,1740,22.42
2,Vias,1697,21.87
3,Aeropuerto,430,5.54
4,Ambiental,426,5.49
5,Educacion,273,3.52
6,Agro,239,3.08
7,Energia,212,2.73
8,Transporte,175,2.25
9,Vivienda,159,2.05



____________________